In [2]:
import numpy as np
import wntr
import networkx as nx

C:\Users\vedik\anaconda3\envs\wdn_gnn\lib\site-packages\wntr\epanet\toolkit.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [4]:
wn = wntr.network.WaterNetworkModel("Net2.inp")

print("Number of junctions:", len(wn.junction_name_list))
print("Total nodes:", wn.num_nodes)
print("Total pipes:", wn.num_links)

Number of junctions: 35
Total nodes: 36
Total pipes: 40


In [5]:
# NetworkX graph
G = wn.get_graph()

# Fixed node order
node_list = list(G.nodes())
node_to_idx = {n: i for i, n in enumerate(node_list)}
num_nodes = len(node_list)

# Edge index (undirected)
edge_index = []
for u, v in G.edges():
    edge_index.append([node_to_idx[u], node_to_idx[v]])
    edge_index.append([node_to_idx[v], node_to_idx[u]])

edge_index = np.array(edge_index).T

print("edge_index shape:", edge_index.shape)

edge_index shape: (2, 80)


C:\Users\vedik\AppData\Local\Temp\ipykernel_24696\2147871450.py:2: DeprecationWarning: wntr.network.WaterNetworkModel.get_graph is deprecated, use wntr.network.WaterNetworkModel.to_graph instead
  G = wn.get_graph()


In [6]:
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

pressure_baseline = results.node["pressure"]
time_steps = pressure_baseline.index.tolist()

# baseline pressure at all times
P_baseline = pressure_baseline.copy()

print("Baseline simulation done")


Baseline simulation done


In [7]:
junction_list = wn.junction_name_list

# Leak magnitudes (you can adjust later)
leak_magnitudes = np.linspace(0.002, 0.04, 8)

print("Leak junctions:", len(junction_list))
print("Leak magnitudes:", leak_magnitudes)
print("Time steps:", time_steps)


Leak junctions: 35
Leak magnitudes: [0.002 0.007 0.013 0.018 0.024 0.029 0.035 0.04 ]
Time steps: [0, 3600, 7200, 10800, 14400, 18000, 21600, 25200, 28800, 32400, 36000, 39600, 43200, 46800, 50400, 54000, 57600, 61200, 64800, 68400, 72000, 75600, 79200, 82800, 86400, 90000, 93600, 97200, 100800, 104400, 108000, 111600, 115200, 118800, 122400, 126000, 129600, 133200, 136800, 140400, 144000, 147600, 151200, 154800, 158400, 162000, 165600, 169200, 172800, 176400, 180000, 183600, 187200, 190800, 194400, 198000]


In [8]:
X = []
y = []

for leak_junc in junction_list:
    for leak_size in leak_magnitudes:
        
        # reload clean network
        wn_leak = wntr.network.WaterNetworkModel("Net2.inp")
        wn_leak.get_node(leak_junc).add_demand(leak_size, "leak")
        
        sim = wntr.sim.EpanetSimulator(wn_leak)
        results = sim.run_sim()
        pressure_leak = results.node["pressure"]
        
        for t in time_steps:
            delta_p = np.abs(
                P_baseline.loc[t] - pressure_leak.loc[t]
            )
            
            X.append(delta_p.loc[node_list].values)
            y.append(junction_list.index(leak_junc))


In [9]:
X = np.array(X)   # (N_samples, num_nodes)
y = np.array(y)   # (N_samples,)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Number of classes:", len(np.unique(y)))


X shape: (15680, 36)
y shape: (15680,)
Number of classes: 35


In [10]:
# Build adjacency list
adj = [[] for _ in range(num_nodes)]
for u, v in edge_index.T:
    adj[u].append(v)

X_aug = []

for sample in X:
    mean_dp = sample.mean()
    rel_dp = sample / (mean_dp + 1e-6)
    
    local_rel = []
    for i in range(num_nodes):
        neigh = adj[i]
        if len(neigh) == 0:
            local_rel.append(0.0)
        else:
            local_rel.append(
                sample[i] / (sample[neigh].mean() + 1e-6)
            )
    
    X_aug.append(
        np.stack([sample, rel_dp, local_rel], axis=1)
    )

X_aug = np.array(X_aug)

print("Augmented X shape:", X_aug.shape)

Augmented X shape: (15680, 36, 3)


In [11]:
np.save("X_net2.npy", X_aug)
np.save("y_net2.npy", y)
np.save("edge_index_net2.npy", edge_index)

print("Net2 dataset saved successfully")

Net2 dataset saved successfully


In [12]:
np.save("node_list_net2.npy", np.array(node_list))
np.save("junction_list_net2.npy", np.array(junction_list))